In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = False

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

T_f = 350.0

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSR_avg%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFDSR_avg%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=T_f)
abTempF2S = InterfaceData(size=inodes, value=T_f)
TempS = InterfaceData(size=inodes, value=T_bot)
tempF = InterfaceData(size=inodes, value=T_f)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF2S = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)[f2s]

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
Nmax = 8000
t = 0.0
avg_pc_iterations = 0.0
flog = open('FDSR_avg%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FDSR_avg%i.log' % k, mode='a')

abtemp_prev = abTempF.curr.copy()
temps_prev = TempS.curr.copy()

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    abtemp_prev[:] = abTempF.curr
    temps_prev[:] = TempS.curr
    
    tempF.curr[:] = TempS.curr[s2f]
    
    solverF.put_temperature_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=tempF.curr
    )
    
    while True:
        # back up previous interface value
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        abTempF2S.curr[:] = 0.5*(abTempF.curr[f2s]+abtemp_prev[f2s])
        # update solid flux interface
        solverS.update_robin_interface(
            tag=interface_tag,
            Ta=abTempF2S.curr,
            h_coeff=hF2S
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        TempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
        
        # mapping
        tempF.curr[:] = TempS.curr[s2f]
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(tempF)
                under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=0.5*(tempF.curr + temps_prev[s2f])
            )
            solverF.restore_previous_solutions()
            solverS.restore()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-T_bot)/(T_f-T_bot)
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=9.
step=1, pc_iterations=4.
step=2, pc_iterations=4.
step=3, pc_iterations=4.
step=4, pc_iterations=3.
step=5, pc_iterations=3.
step=6, pc_iterations=4.
step=7, pc_iterations=3.
step=8, pc_iterations=3.
step=9, pc_iterations=3.
step=10, pc_iterations=3.
step=11, pc_iterations=3.
step=12, pc_iterations=3.
step=13, pc_iterations=3.
step=14, pc_iterations=3.
step=15, pc_iterations=3.
step=16, pc_iterations=3.
step=17, pc_iterations=3.
step=18, pc_iterations=3.
step=19, pc_iterations=3.
step=20, pc_iterations=3.
step=21, pc_iterations=3.
step=22, pc_iterations=3.
step=23, pc_iterations=3.
step=24, pc_iterations=3.
step=25, pc_iterations=3.
step=26, pc_iterations=2.
step=27, pc_iterations=2.
step=28, pc_iterations=3.
step=29, pc_iterations=3.
step=30, pc_iterations=2.
step=31, pc_iterations=2.
step=32, pc_iterations=2.
step=33, pc_iterations=2.
step=34, pc_iterations=2.
step=35, pc_iterations=2.
step=36, pc_iterations=2.
step=37, pc_iterations=2.
step=38, pc_iterations

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-T_bot)/(T_f-T_bot)
print(theta)
sio.savemat(
    'numeric_res/kFDSR_avg%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.06309362 0.06174974 0.0596082  0.05753227 0.05557319 0.05381487
 0.05217191 0.05065481 0.04923569 0.04786381 0.04658148 0.04535607
 0.0441976  0.0430825  0.04202552 0.04100853 0.04004005 0.03910454
 0.03820762 0.03734904 0.03652137 0.03572771 0.03496001 0.03422392
 0.03351641 0.03283658 0.03217753 0.0315443  0.03093172 0.03034196
 0.02977463 0.02922656 0.02869762 0.0281895  0.02769521 0.02722098
 0.02676347 0.02632371 0.02589641 0.02548561 0.02508842 0.02470507
 0.02433638 0.02398068 0.02363778 0.02330715 0.02298889 0.02268155
 0.02238683 0.02210202 0.02183018 0.02156675 0.02131473 0.02107353
 0.0208413  0.02062036 0.02040766 0.02020587 0.02001333 0.01982979
 0.01965653 0.01949143 0.01933746 0.01919085 0.01905488 0.01892828
 0.01881019 0.01870291 0.01860433 0.01851589 0.01843759 0.01836846
 0.01831144 0.01826305 0.01822739 0.01820124 0.01818828 0.01818651
 0.01819671 0.01822098 0.0182575  0.01830943 0.01837587 0.01845693
 0.0185556  0.01867131 0.01880697 0.01896157 0.01913856 0.0193